In [16]:
from datetime import datetime, timedelta
import pandas as pd
import pickle

def julian_to_datetime(julian_date):
    #Julian Date for Unix epoch (1970-01-01)
    JULIAN_EPOCH = 2440587.5
    try:
        # Calculate the number of days from the Unix epoch
        days_from_epoch = julian_date - JULIAN_EPOCH
        # Create a datetime object for the Unix epoch and add the calculated days
        unix_epoch = datetime(1970, 1, 1)
        result_date = unix_epoch + timedelta(days=days_from_epoch)
        return result_date
    except OverflowError as e:
        # Handle dates that are out of range
        print(f"Date conversion error: {e}")
        return None


def find_mass_bin(mass, scen_properties, species_cell):
    """
    Find the mass bin for a given mass.

    :param mass: Mass of the object in kg
    :type mass: float
    :param scen_properties: The scenario properties object
    :type scen_properties: ScenarioProperties
    :param species_cell: The species cell to find the mass bin for
    :type species_cell: Species
    :return: The mass bin for the given mass
    :rtype: int
    """
    for species in species_cell:
        if species.mass_lb <= mass < species.mass_ub:
            return species.sym_name
    return None

def find_alt_bin(altitude, scen_properties):
    """
    Given an altidude and the generic pySSEM properties, it will calculate the index from the R02 array

    :param altitude: Altitude of an object
    :type altitude: int
    :param scen_properties: The scenario properties object
    :type scen_properties: ScenarioProperties
    :return: Orbital Shell Array Index or None if out of range
    :rtype: int
    """
    shell_altitudes = scen_properties.R0_km

    # The case for an object where it is below the lowest altitude
    if altitude < shell_altitudes[0]:
        return
    
    # The case for an object where it is above the highest altitude
    if altitude >= shell_altitudes[-1]:
        return 

    for i in range(len(shell_altitudes)):  # -1 to prevent index out of range
        try:
            if shell_altitudes[i] <= altitude < shell_altitudes[i + 1]:
                return i  
        except IndexError: # This is the top most shell and will be the last one
            return len(shell_altitudes) 
    

def define_object_class(T):
    """
    Define the object class of each object in the traffic model.
    Adds them to a new column named "obj_type" or overwrites the existing column.

    :param T: list of launches
    :type T: pandas.DataFrame
    """

    T['obj_class'] = "Unknown"

    # Classify Rocket Bodies
    T.loc[T['obj_type'] == 1, 'obj_class'] = "Rocket Body"

    # Classify Satellites
    T.loc[(T['obj_type'] == 2) & (T['stationkeeping'] != 0) & (T['stationkeeping'] < 5), 'obj_class'] = "Station-keeping Satellite"
    T.loc[(T['obj_type'] == 2) & (T['stationkeeping'] == 0), 'obj_class'] = "Non-station-keeping Satellite"
    T.loc[(T['obj_type'] == 2) & (T['stationkeeping'] == 5), 'obj_class'] = "Coordinated Satellite"
    T.loc[(T['obj_type'] == 2) & (T['stationkeeping'] == 6), 'obj_class'] = "Candidate Satellite"

    # Classify Debris
    T.loc[T['obj_type'].isin([3, 4]), 'obj_class'] = "Debris"

    # Count unclassified rows
    unclassed_rows = (T['obj_class'] == "Unknown").sum()
    if unclassed_rows > 0:
        print(f'\t{unclassed_rows} Unclassified rows remain.')

    return T

with open(r'D:\ucl\pyssem\scenario_properties_long.pkl', 'rb') as f:
    scen_properties = pickle.load(f)

In [17]:

# Load the traffic model data
T = pd.read_csv(r"D:\ucl\pyssem\src\pyssem\utils\launch\data\x0_launch_repeatlaunch_2018to2022_megaconstellationLaunches_Constellations.csv")

T['epoch_start_datime'] = T['epoch_start'].apply(lambda x: julian_to_datetime(x))

if 'obj_class' not in T.columns:
    T = define_object_class(T)  # Make sure this function is defined and imported

# Calculate Apogee, Perigee, and Altitude
T['apogee'] = T['sma'] * (1 + T['ecc'])
T['perigee'] = T['sma'] * (1 - T['ecc'])
T['alt'] = (T['apogee'] + T['perigee']) / 2 - scen_properties.re

# Map species type based on object class
species_dict = {"Non-station-keeping Satellite": "sns",
                "Rocket Body": "B",
                "Station-keeping Satellite": "Su",
                "Coordinated Satellite": "S",
                "Debris": "N",
                "Candidate Satellite": "C"}

T['species_class'] = T['obj_class'].map(species_dict)

# Initialize an empty DataFrame for new data
T_new = pd.DataFrame()

# Loop through object classes and assign species based on mass
for obj_class in T['obj_class'].unique():
        species_class = species_dict.get(obj_class)
        if species_class in scen_properties.species_cells:
                if len(scen_properties.species_cells[species_class]) == 1:
                        T_obj_class = T[T['obj_class'] == obj_class].copy()
                        T_obj_class['species'] = scen_properties.species_cells[species_class][0].sym_name
                        T_new = pd.concat([T_new, T_obj_class])
                else:
                        species_cells = scen_properties.species_cells[species_class]
                        T_obj_class = T[T['obj_class'] == obj_class].copy()
                        T_obj_class['species'] = T_obj_class['mass'].apply(find_mass_bin, args=(scen_properties, species_cells)) 
                        T_new = pd.concat([T_new, T_obj_class])

# Assign objects to corresponding altitude bins
T_new['alt_bin'] = T_new['alt'].apply(find_alt_bin, args=(scen_properties,))

# length of alt bin
print(f"Length of Alt Bin: {len(T_new['alt_bin'].unique())}")

# Filter T_new to include only species present in scen_properties
T_new = T_new[T_new['species_class'].isin(scen_properties.species_cells.keys())]

# Initial population
x0 = T_new[T_new['epoch_start_datime'] < scen_properties.start_date]

print(x0['alt_bin'].unique())


# Create a pivot table, keep alt_bin
df = x0.pivot_table(index='alt_bin', columns='species', aggfunc='size', fill_value=0)

# Create a new data frame with column names like scenario_properties.species_sym_names and rows of length n_shells
x0_summary = pd.DataFrame(index=range(scen_properties.n_shells), columns=scen_properties.species_names).fillna(0)
x0_summary.index.name = 'alt_bin'

# Merge the two dataframes
for column in df.columns:
    if column in x0_summary.columns:
        x0_summary[column] = df[column]

# fill NaN with 0
x0_summary.fillna(0, inplace=True)

# Future Launch Model
flm_steps = pd.DataFrame()

time_increment_per_step = scen_properties.simulation_duration / scen_properties.steps

time_steps = [scen_properties.start_date + timedelta(days=365.25 * time_increment_per_step * i) 
            for i in range(scen_properties.steps + 1)]    

for start, end in zip(time_steps[:-1], time_steps[1:]):
    flm_step = T_new[(T_new['epoch_start_datime'] >= start) & (T_new['epoch_start_datime'] < end)]
    #print(f"Step: {start} - {end}, Objects: {flm_step.shape[0]}")
    flm_summary = flm_step.groupby(['alt_bin', 'species']).size().unstack(fill_value=0)

    # all objects aren't always in shells, so you need to these back in. 
    flm_summary = flm_summary.reindex(range(0, scen_properties.n_shells), fill_value=0)

    flm_summary.reset_index(inplace=True)
    flm_summary.rename(columns={'index': 'alt_bin'}, inplace=True)

    flm_summary['epoch_start_date'] = start # Add the start date to the table for reference
    flm_steps = pd.concat([flm_steps, flm_summary])



# [x0, flm] = ADEPT_traffic_model(scenario_properties, r"D:\ucl\pyssem\src\pyssem\utils\launch\data\x0_launch_repeatlaunch_2018to2022_megaconstellationLaunches_Constellations.csv")

Length of Alt Bin: 41
[nan 12. 14. 19. 17. 29. 21. 24. 18. 27. 16. 13. 26. 30. 15. 23. 20. 28.
 25. 22. 36. 31. 37. 39. 34. 33. 35.  8. 38.  7. 32. 10.  9. 11.  6.  1.
  5.  4.  3.  2.  0.]


In [15]:
x0

,Su_260kg,Su_473kg,S_148kg,S_750kg,S_1250kg,sns,N_0.00141372kg,N_0.567kg,N_6kg,N_148kg,N_260kg,N_473kg,N_750kg,N_1250kg,B
alt_bin,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,11,0,0,0,0,1,0,0,0,0,0,0,0,0,3
3,14,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,30,0,0,0,0,16,0,0,0,0,0,0,0,0,2
5,43,0,0,0,0,9,0,0,0,0,0,0,0,0,2
6,44,0,0,0,0,11,0,0,0,0,0,0,0,0,7
7,117,0,0,0,0,7,0,0,0,0,0,0,0,0,10
8,161,0,0,0,0,17,0,0,0,0,0,0,0,0,17
